In [4]:
import numpy as np
import sounddevice as sd
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import audio
import time

In [6]:
print(sd.query_devices())

  0 BlackHole 16ch, Core Audio (16 in, 16 out)
< 1 Externe Kopfhörer, Core Audio (0 in, 2 out)
> 2 MacBook Pro-Mikrofon, Core Audio (1 in, 0 out)
  3 MacBook Pro-Lautsprecher, Core Audio (0 in, 2 out)
  4 Microsoft Teams Audio, Core Audio (2 in, 2 out)
  5 ZoomAudioDevice, Core Audio (2 in, 2 out)
  6 BH-1-2, Core Audio (16 in, 16 out)
  7 BH-3-4, Core Audio (16 in, 16 out)
  8 Multi-Output Device, Core Audio (0 in, 2 out)


In [ ]:
AudioClassifier = mp.tasks.audio.AudioClassifier
AudioClassifierOptions = mp.tasks.audio.AudioClassifierOptions
AudioClassifierResult = mp.tasks.audio.AudioClassifierResult
AudioRunningMode = mp.tasks.audio.RunningMode
BaseOptions = mp.tasks.BaseOptions

def print_result(result: AudioClassifierResult, timestamp_ms: int):
    print('AudioClassifierResult result: {}'.format(result))

options = AudioClassifierOptions(
    base_options=BaseOptions(model_asset_path='yamnet-classification.tflite'),
    running_mode=AudioRunningMode.AUDIO_STREAM,
    max_results=5,
    result_callback=print_result)

# with AudioClassifier.create_from_options(options) as classifier:
#   # The classifier is initialized. Use it here.
#   # ...
#   print('something')


# Send live audio data to perform audio classification.
# Results are sent to the `result_callback` provided in the `AudioClassifierOptions`
# classifier.classify_async(audio_data, timestamp_ms)
# Create the audio classifier
classifier = AudioClassifier.create_from_options(options)
  
### Prepare the data

In [8]:
AudioData = mp.tasks.components.containers.AudioData


# Parameters
fs = 44100.0  # Sample rate
sample_rate = 16000
channels = 1  # Mono audio
block_size = 256  # Number of samples per callback
device_index = 2  # Replace with your desired input device index

# Read microphone data as np arrays, then call

# Callback function to process audio in real-time
def audio_callback(indata, frames, time, status):
    if status:
        print(status, flush=True)  # Handle errors
    print("Received block with shape:", indata.shape)  # Process audio here
    # Flatten the input data and convert to a list
    # audio_data = np.reshape(indata, -1).tolist()
    # Get the current timestamp in milliseconds
    timestamp_ms = int(time.inputBufferAdcTime * 1000)
    # Create an AudioData object
    # mp_audio_data = AudioData(
    #   buffer=audio_data,
    #   sample_rate=sample_rate  # Adjust based on your model requirements
    # )
    # mp_audio_data = AudioData(
    #     buffer=audio_data,
    #     sample_rate=sample_rate
    # )
    # Send the audio data to the classifier
    # classifier.classify_async(mp_audio_data, timestamp_ms)
    # audio_data = AudioData.create_from_array(indata.astype(float) / np.iinfo(np.int16).max, fs)
    # Get the current timestamp in milliseconds
    # timestamp_ms = int(time.inputBufferAdcTime * 1000)
    # classifier.classify_async(audio_data, timestamp_ms)
    # time.sleep(0.9)
    # Convert NumPy array to a raw byte buffer
    # audio_data_bytes = np.array(indata, dtype=np.float32).tobytes()

    # # Create AudioData instance
    # mp_audio_data = AudioData.create_from_array(audio_data_bytes, sample_rate=sample_rate)
    # Ensure audio data is in float32 format (required by MediaPipe)
    audio_data_np = np.array(indata, dtype=np.float32)

    # Create AudioData instance correctly
    mp_audio_data = AudioData.create_from_array(audio_data_np, sample_rate=sample_rate)
    classifier.classify_async(mp_audio_data, timestamp_ms)

# Open a stream with the selected device
with sd.InputStream(samplerate=sample_rate, channels=channels, callback=audio_callback,blocksize=block_size, device=6):
    print("Streaming from device:", device_index)
    print("Press Ctrl+C to stop.")
    try:
      while True:
        time.sleep(0.9)
        pass  # Keep the stream open
    except KeyboardInterrupt:
      print('stpped!')


    

: 